In [1]:
 from sklearn import tree
import pandas as pd
import os
import numpy as np

In [2]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [14]:
selected_features = df[['koi_disposition', 'koi_period', 'koi_time0bk', 'koi_slogg', 'koi_srad', 'ra', 'dec', 'koi_kepmag' ]]
selected_features.head()

,koi_disposition,koi_period,koi_time0bk,koi_slogg,koi_srad,ra,dec,koi_kepmag
0,CONFIRMED,54.418383,162.513840,4.467,0.927,291.93423,48.141651,15.347
1,FALSE POSITIVE,19.899140,175.850252,4.544,0.868,297.00482,48.134129,15.436
2,FALSE POSITIVE,1.736952,170.307565,4.564,0.791,285.53461,48.285210,15.597
3,CONFIRMED,2.525592,171.595550,4.438,1.046,288.75488,48.226200,15.509
4,CONFIRMED,4.134435,172.979370,4.486,0.972,296.28613,48.224670,15.714


# Create a Train Test Split
Use koi_disposition for the y values

In [26]:
target = selected_features.koi_disposition
data = selected_features.drop("koi_disposition", axis = 1)
feature_names = data.columns
data.head()



,koi_period,koi_time0bk,koi_slogg,koi_srad,ra,dec,koi_kepmag
0,54.418383,162.513840,4.467,0.927,291.93423,48.141651,15.347
1,19.899140,175.850252,4.544,0.868,297.00482,48.134129,15.436
2,1.736952,170.307565,4.564,0.791,285.53461,48.285210,15.597
3,2.525592,171.595550,4.438,1.046,288.75488,48.226200,15.509
4,4.134435,172.979370,4.486,0.972,296.28613,48.224670,15.714


In [27]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data,target,random_state=42)

In [28]:
X_train.head()

,koi_period,koi_time0bk,koi_slogg,koi_srad,ra,dec,koi_kepmag
6122,6.768901,133.077240,4.327,1.125,294.40472,39.351681,14.725
6370,0.733726,132.020050,4.578,0.797,284.50391,42.463860,15.770
2879,7.652707,134.460380,4.481,0.963,295.50211,38.983540,13.099
107,7.953547,174.662240,4.536,0.779,291.15878,40.750271,15.660
29,4.959319,172.258529,4.359,1.082,292.16705,48.727589,15.263


In [29]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.4805491990846682

In [30]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.5835240274599542

In [31]:
sorted(zip(rf.feature_importances_, feature_names), reverse=True)

[(0.19615900714084067, 'koi_period'),
 (0.15858441552451324, 'koi_time0bk'),
 (0.13716780980492954, 'ra'),
 (0.13027524035930607, 'dec'),
 (0.12720700349760447, 'koi_srad'),
 (0.1256087500625085, 'koi_kepmag'),
 (0.12499777361029746, 'koi_slogg')]

# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [43]:
# Create the GridSearchCV model
from sklearn.model_selection import GridSearchCV
param_grid = {'criterion': ["gini", "entropy"],
              'max_depth' : np.arange(3,10)}
grid = GridSearchCV(rf, param_grid, verbose=3)

In [44]:
# Train the model with GridSearch
grid.fit(X_train, y_train)

Fitting 5 folds for each of 14 candidates, totalling 70 fits
[CV] criterion=gini, max_depth=3 .....................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ......... criterion=gini, max_depth=3, score=0.528, total=   0.6s
[CV] criterion=gini, max_depth=3 .....................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s remaining:    0.0s


[CV] ......... criterion=gini, max_depth=3, score=0.545, total=   0.6s
[CV] criterion=gini, max_depth=3 .....................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.2s remaining:    0.0s


[CV] ......... criterion=gini, max_depth=3, score=0.532, total=   0.6s
[CV] criterion=gini, max_depth=3 .....................................
[CV] ......... criterion=gini, max_depth=3, score=0.535, total=   0.6s
[CV] criterion=gini, max_depth=3 .....................................
[CV] ......... criterion=gini, max_depth=3, score=0.537, total=   0.7s
[CV] criterion=gini, max_depth=4 .....................................
[CV] ......... criterion=gini, max_depth=4, score=0.546, total=   0.8s
[CV] criterion=gini, max_depth=4 .....................................
[CV] ......... criterion=gini, max_depth=4, score=0.572, total=   0.7s
[CV] criterion=gini, max_depth=4 .....................................
[CV] ......... criterion=gini, max_depth=4, score=0.560, total=   0.8s
[CV] criterion=gini, max_depth=4 .....................................
[CV] ......... criterion=gini, max_depth=4, score=0.564, total=   0.7s
[CV] criterion=gini, max_depth=4 .....................................
[CV] .

[CV] ...... criterion=entropy, max_depth=8, score=0.564, total=   1.7s
[CV] criterion=entropy, max_depth=8 ..................................
[CV] ...... criterion=entropy, max_depth=8, score=0.578, total=   1.8s
[CV] criterion=entropy, max_depth=8 ..................................
[CV] ...... criterion=entropy, max_depth=8, score=0.583, total=   1.7s
[CV] criterion=entropy, max_depth=8 ..................................
[CV] ...... criterion=entropy, max_depth=8, score=0.579, total=   1.8s
[CV] criterion=entropy, max_depth=8 ..................................
[CV] ...... criterion=entropy, max_depth=8, score=0.586, total=   1.7s
[CV] criterion=entropy, max_depth=9 ..................................
[CV] ...... criterion=entropy, max_depth=9, score=0.561, total=   1.8s
[CV] criterion=entropy, max_depth=9 ..................................
[CV] ...... criterion=entropy, max_depth=9, score=0.584, total=   1.8s
[CV] criterion=entropy, max_depth=9 ..................................
[CV] .

[Parallel(n_jobs=1)]: Done  70 out of  70 | elapsed:  1.4min finished


GridSearchCV(estimator=RandomForestClassifier(n_estimators=200),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': array([3, 4, 5, 6, 7, 8, 9])},
             verbose=3)

In [45]:
print(grid.best_params_)
print(grid.best_score_)


{'criterion': 'entropy', 'max_depth': 8}
0.578105829615992


# Save the Model

In [46]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'model_2.sav'
joblib.dump(clf, filename)

['model_2.sav']